# Importando as bibliotecas





In [1]:
import tensorflow as tf
tf.test.is_built_with_cuda()
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)
tf.config.list_physical_devices('GPU')


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8680792739478492068
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2252026676
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2492814721059881620
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
gpus = tf.config.list_logical_devices('GPU')

type(gpus)
if gpus:
  # Replicate your computation on multiple GPUs
  c = []
  for gpu in gpus:
    with tf.device(gpu.name):
        print(gpu.name)
        pass
gpu = gpus[0]

/device:GPU:0


In [4]:
import sys
import os
from sklearn.model_selection import train_test_split
sys.path.append('./')
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import ThresholdedReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import models
import h5py
from tensorflow.keras.models import Model
from keras import Input
from tensorflow.keras import optimizers
import json
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
import sklearn.metrics as sklm
import pandas as pd

In [5]:
CHARNN_DATA_CONFIG = "D:/SiDi/Project/Modulo II/github/multimodal-sentiment-analysis/rbsa2/charnn_cnn/charcnn_data_config.json"
charcnn_data_config = pd.read_json(CHARNN_DATA_CONFIG)

CNN_DATA_CONFIG = "D:/SiDi/Project/Modulo II/github/multimodal-sentiment-analysis/rbsa2/charnn_cnn/nn_config.json"
nn_data_config = pd.read_json(CNN_DATA_CONFIG)


In [6]:
#data_config = json.load(open("charcnn_data_config.json")) #Leitura do arquivo de configuracao
data_config = charcnn_data_config #Leitura do arquivo de configuracao

#nn_config = json.load(open("nn_config.json")) # Leitura de arquivo de configuracao, quais sao os parametros
nn_config = nn_data_config # Leitura de arquivo de configuracao, quais sao os parametros

alphabet_size = data_config["data"]["alphabet_size"]+1 #tamanho do alfabeto
input_size = data_config["data"]["input_size"] #Tamanho do input de zi

fully_connected_layers = nn_config["char_cnn_zhang"]["fully_connected_layers"] #Camadas Conectadas
conv_layers = nn_config["char_cnn_zhang"]["conv_layers"] #Camadas Convulacionasi
threshold = nn_config["char_cnn_zhang"]["threshold"] # Limiar
dropout_p = nn_config["char_cnn_zhang"]["dropout_p"] #Taxa de Dropout
embedding_size = nn_config["char_cnn_zhang"]["embedding_size"] # Tamnho do Embedding

In [7]:
DATASET_PATH_URL  = "D:/SiDi/Project/Modulo II/dataset/dataset_sidi_512.csv"
df = pd.read_csv(DATASET_PATH_URL, sep='\t')

print(len(df))
df.reset_index(inplace=True)

12000


In [8]:
df.head()

,index,Unnamed: 0,Unnamed: 0.1,image_name,NEG,NEU,POS,text,sent_text,sent_image,image_path,image_height,image_width
0,0,382583,382583,798338609870872577-2,0.774375,0.174701,0.050924,what does it look like i do for a living? (cra...,NEG,0,data/79833/798338609870872577-2.jpg,1280,722
1,1,443004,443004,802556641057054721-1,0.540009,0.304103,0.155888,No cheat just skill. #ClikerHeroes https://t.c...,NEG,0,data/80255/802556641057054721-1.jpg,707,1366
2,2,348885,348885,796032212407721984-1,0.513661,0.322456,0.163883,@KEILOin_DaTrunk I deleted https://t.co/qIhBkn...,NEG,0,data/79603/796032212407721984-1.jpg,1334,750
3,3,377487,377487,798011160532381696-1,0.712648,0.254173,0.033180,"""RT NYTFashion: How Nasty Gal went from an eBa...",NEG,0,data/79801/798011160532381696-1.jpg,561,1000
4,4,423703,423703,801181521679790080-1,0.713677,0.186854,0.099468,I hate this nigga lmaooo https://t.co/gPcyJfJESN,NEG,0,data/80118/801181521679790080-1.jpg,1334,750


In [9]:
# Verifica o balanceamento das classes
classes = df.groupby('sent_image')['index'].nunique()/df.shape[0] #Agrupe por index
classes

sent_image
0    0.333333
1    0.333333
2    0.333333
Name: index, dtype: float64

In [10]:
image_height = 128 #Defina o tamanho  da imagem
image_width = 128 #Defina a largura da iamge
from tensorflow.keras.preprocessing.image import load_img #carrega funcao para carraegar imagem
from tensorflow.keras.preprocessing.image import img_to_array #carrega funcao para converter imagem para array
from PIL import Image
#Funcao que remove quebra de linha
def remove_spaces(cell):  
    return cell.replace('\n',' ')
#Converte que carrega a image e depois convertte para array
def get_array(image_path):
    img = load_img("D:\\SiDi\\Project\\Modulo II\\dataset\\512" + '/' + f"{image_path}.jpg", target_size=(image_height, image_width), color_mode='grayscale')

    return img_to_array(img)


#Mapeaa os tipos de classes existente nos rotulos de predicoes
def to_one_hot(y, class_mapping, num_classes):
    y_int = y.map(class_mapping)
    return to_categorical(y_int, num_classes)

#X_text = df['text'].apply(remove_spaces)


# Numero de classes do dataset

In [11]:

num_classes = df['sent_text'].nunique()
#Faz o mapeeamento da classe com o id
class_mapping = {label:idx for idx,label in enumerate(np.unique(df['sent_image']))}
class_mapping
#y_int = df['sent_text'].map(class_mapping)

{0: 0, 1: 1, 2: 2}

In [12]:
X_train_text = []
X_train_image = []
y_train = []
image_path = []
y_total = []
img_total = []
text_total = []
page_total = []
doc_total = []
target_total = []

for i in range(1,len(df[:9600]),1):
    X_image = df[i-1:i]['image_name'].apply(get_array)
    X_text = df[i-1:i]['text']
    y = to_one_hot(df[ i-1:i]['sent_image'], class_mapping, num_classes)
    
    text_image = pd.concat([X_image, X_text], axis=1)
    
    image_path.extend( df[i-1:i]['image_path'])
    #X_train_text.extend(X_text)
    X_train_image.extend(X_image)
    y_train.extend(y)
    y_total.append(y)
    #text_total.append(X_text)
    img_total.append(X_image)
    page_total.append( df[i-1:i]['image_name'])
    doc_total.append( df[i-1:i]['image_name'])
    target_total.append( df[i-1:i]['sent_image'])
    
    
    
#X_train_text = pd.Series(X_train_text)
X_train_image = pd.Series(X_train_image)
y_train = np.array(y_train)


In [13]:
len(y_total)
len(text_total)
len(img_total)

9599

In [14]:
X_test_text = []
X_test_image = []
y_test = []

image_path = []
y_total = []
img_total = []
text_total = []
page_total = []
doc_total = []
target_total = []

for i in range(1,len(df[9601:]),1):
    X_image = df[i-1:i]['image_name'].apply(get_array)
    y = to_one_hot(df[i-1:i]['sent_image'], class_mapping, num_classes)
    
    #text_image = pd.concat([X_image, X_text], axis=1)
    
    #X_test_text.extend(X_text)
    X_test_image.extend(X_image)
    y_test.extend(y)
    y_total.append(y)
    #text_total.append(X_text)
    img_total.append(X_image)
    page_total.append(df[i-1:i]['image_name'])
    doc_total.append(df[i-1:i]['image_name'])
    target_total.append(df[i-1:i]['sent_image'])
    
    
#X_test_text = pd.Series(X_test_text)
X_test_image = pd.Series(X_test_image)
y_test = np.array(y_test)

In [15]:
len(y_total), len(img_total)

(2398, 2398)

In [16]:
class_mapping

{0: 0, 1: 1, 2: 2}

# Join Train and Test (Test and Image)

In [17]:
X_join_image = X_train_image.append(X_test_image)
len(X_join_image)

11997

# Converte de Imagem para Array Numpy - image_to_numpy(image_df)

In [18]:
def image_to_numpy(image_df):
    all_images = []
    for i in range(len(image_df)):
        all_images.append(image_df.iloc[i])  
    return np.array(all_images)

X_train_image = image_to_numpy(X_train_image)
X_test_image = image_to_numpy(X_test_image)

In [19]:

X_join_image = image_to_numpy(X_join_image)

# Imprime o Shape do Objeto de Treinamento Convertido Array Numpy 

In [20]:
print(X_train_image.shape)

(9599, 128, 128, 1)


# (01) - Conjunto de Dados de Validacao - Declara a Classe ImageValidationCheckpoint

In [21]:
def predict(model, X_test_image, batch_size=32):
    y_predict = np.round(model.predict(X_test_image))
    return y_predict

def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

class ImageValidationCheckpoint(Callback):
    def __init__(self, filepath, X_test_image,y_test, metric = 'kappa'):
        self.X_test_image = X_test_image
        self.y_test = y_test
        self.metric = metric
        self.max_metric = float('-inf')
        self.max_metrics = None
        self.filepath = filepath
        self.history = []

    def on_epoch_end(self, epoch, logs={}):
        
        predicted_labels = predict(self.model, X_test_image)
        #true_labels = [label2Idx[x[1]] for x in self.test_data]
        true_labels = one_hot_decode(self.y_test)
        predicted_labels = one_hot_decode(predicted_labels)
        eval_metrics = {
            'accuracy' : sklm.accuracy_score(true_labels, predicted_labels),
            'f1_micro' : sklm.f1_score(true_labels, predicted_labels, average='micro'),
            'f1_macro' : sklm.f1_score(true_labels, predicted_labels, average='macro'),
            'f1_binary' : sklm.f1_score(true_labels, predicted_labels, average='weighted', pos_label = 1),
            'kappa' : sklm.cohen_kappa_score(true_labels, predicted_labels)
        }
        eval_metric = eval_metrics[self.metric]
        self.history.append(eval_metric)
        
        if epoch > -1 and eval_metric > self.max_metric:
            print("\n" + self.metric + " improvement: " + str(eval_metric) + " (before: " + str(self.max_metric) + "), saving to " + self.filepath)
            self.max_metric = eval_metric     # optimization target
            self.max_metrics = eval_metrics   # all metrics
            #self.model.save(self.filepath)

image_input = Input(shape=(image_height, image_width, 1), name='image_input', dtype='float32')
x = layers.Conv2D(32, (8, 8), activation='relu',input_shape=(image_height, image_width, 1))(image_input)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(32, (6, 6), activation='relu')(x)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(32, (8, 8), activation='relu',input_shape=(image_height, image_width, 1))(image_input)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(32, (6, 6), activation='relu')(x)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Flatten()(x)
encoded_image = layers.Dense(256, activation='relu', name='encoded_image')(x)

output = layers.Dense(num_classes, activation='softmax', name='output')(encoded_image)
model_image = Model(image_input, output)
model_image.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.RMSprop(lr=1e-4),
                    metrics=['acc'])
model_image.summary()



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image_input (InputLayer)    [(None, 128, 128, 1)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 121, 121, 32)      2080      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 20, 20, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 15, 15, 32)        36896     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 2, 2, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 128)               0     

D:\qualifacti\caes-vs-gatos\caes-vs-gatos\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [22]:
X_train_image.shape

(9599, 128, 128, 1)

In [23]:
y_train.shape

(9599, 3)

In [24]:
X_test_image.shape

(2398, 128, 128, 1)

In [25]:
y_test.shape

(2398, 3)

In [26]:
n_repeats = 1
exp_history = []
optimize_for = 'accuracy'
model_file = 'v0.0.1_image-tweet-sent.h5'
with tf.device(gpu.name):
    for i in range(n_repeats):
        checkpoint = ImageValidationCheckpoint(model_file, X_test_image, y_test, metric=optimize_for)
        model_image.fit(X_train_image, y_train, epochs=18, batch_size=32,
                         validation_data=(X_test_image, y_test)
                         ,callbacks = [checkpoint])
        exp_history.append(checkpoint.max_metrics)
    


Epoch 1/18
300/300 [==============================] - ETA: 0s - loss: 2.2003 - acc: 0.3962
accuracy improvement: 0.7756463719766472 (before: -inf), saving to v0.0.1_image-tweet-sent.h5
300/300 [==============================] - 29s 81ms/step - loss: 2.2003 - acc: 0.3962 - val_loss: 0.7896 - val_acc: 0.6493
Epoch 2/18
300/300 [==============================] - ETA: 0s - loss: 1.1476 - acc: 0.4086
accuracy improvement: 0.872393661384487 (before: 0.7756463719766472), saving to v0.0.1_image-tweet-sent.h5
300/300 [==============================] - 22s 73ms/step - loss: 1.1476 - acc: 0.4086 - val_loss: 0.7469 - val_acc: 0.7185
Epoch 3/18
300/300 [==============================] - ETA: 0s - loss: 1.0606 - acc: 0.4307
accuracy improvement: 0.9516263552960801 (before: 0.872393661384487), saving to v0.0.1_image-tweet-sent.h5
300/300 [==============================] - 23s 77ms/step - loss: 1.0606 - acc: 0.4307 - val_loss: 0.8388 - val_acc: 0.6939
Epoch 4/18
300/300 [==============================

ResourceExhaustedError:  OOM when allocating tensor with shape[32,32,15,15] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/model/max_pooling2d_3/MaxPool/MaxPoolGrad
 (defined at D:\qualifacti\caes-vs-gatos\caes-vs-gatos\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:464)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_992]

Errors may have originated from an input operation.
Input Source operations connected to node gradient_tape/model/max_pooling2d_3/MaxPool/MaxPoolGrad:
In[0] model/conv2d_3/Relu (defined at D:\qualifacti\caes-vs-gatos\caes-vs-gatos\lib\site-packages\keras\backend.py:4867)	
In[1] model/max_pooling2d_3/MaxPool (defined at D:\qualifacti\caes-vs-gatos\caes-vs-gatos\lib\site-packages\keras\layers\pooling.py:357)	
In[2] 

In [27]:
avg_result = sum([m['kappa'] for m in exp_history]) / n_repeats
avg_acc = sum([m['accuracy'] for m in exp_history]) / n_repeats
print(avg_result)
print(avg_acc)
for i, r in enumerate(exp_history):
    model_file = "ness_exp1_single-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + str(r['f1_macro']) + ' ' + model_file)


0.0
0.0


In [28]:
model_image.save('v0.0.1_32-relu-bt4sa-128-image-tweet-sent.h5')

In [31]:
import time
start = time.time()
y_pred = model_image.predict(X_test_image)

print(time.time()-start)

2.1711220741271973


In [32]:
id_class_mapping = {idx:label for idx,label in enumerate(np.unique(df['sent_image']))}
id_class_mapping

{0: 0, 1: 1, 2: 2}

In [34]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index

pred_class = []
for i in range(len(y_pred)):
  pred = np.where(y_pred[i] == np.amax(y_pred[i]))
  pred_class.append(id_class_mapping[pred[0][0]])
    
print(classification_report(Y_test, pred_class))

              precision    recall  f1-score   support

           0       1.00      0.56      0.72      2398
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0

    accuracy                           0.56      2398
   macro avg       0.33      0.19      0.24      2398
weighted avg       1.00      0.56      0.72      2398



D:\qualifacti\caes-vs-gatos\caes-vs-gatos\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\qualifacti\caes-vs-gatos\caes-vs-gatos\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\qualifacti\caes-vs-gatos\caes-vs-gatos\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
